In [1]:
dbutils.widgets.text("table", "","")
dbutils.widgets.get("table")
table = getArgument("table")

dbutils.widgets.text("backup_folder", "","")
dbutils.widgets.get("backup_folder")
backup_folder = getArgument("backup_folder")

print ("Param -\'input':")
print (table)
print(backup_folder)
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", "f55ddaf6-8ebf-4474-aa00-4b06d8479c61")
spark.conf.set("dfs.adls.oauth2.credential", "9KlkgWv9WMqoDatfLINVLuJMsa057qLob+bdOj1aEv4=")
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/ae5ab930-6b7b-4b0c-9a92-dcc7e8008cb9/oauth2/token ")

Param -'input':
payroll_master.PAY34T
PAY34T

In [2]:
spark.sql("REFRESH TABLE " + table)

Out[ 89 ]: DataFrame[]

In [3]:
import time
import datetime

timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H_%M_%S')

file_path = "adl://linxccdls02.azuredatalakestore.net/Backup/{0}/tmp/".format(backup_folder)
folder_path = "adl://linxccdls02.azuredatalakestore.net/Backup/{0}/".format(backup_folder)


#change to enriched view
master_df = spark.sql("SELECT * FROM " + table);
master_df.repartition(1).write.mode('overwrite').format('csv').save(file_path, header = 'true')

list_of_files = dbutils.fs.ls(file_path)
csv_path = [file.name for file in list_of_files if file.name.endswith(".csv")][0]

print(csv_path)

dbutils.fs.mv(file_path + csv_path, folder_path + timestamp + ".csv")
dbutils.fs.rm(file_path, True)

part-00000-tid-3448243796918907096-556b6719-ab3a-4584-840e-bf4d3eaa3017-203341-c000.csv
 Out[ 90 ]: True

In [4]:
from azure.datalake.store import core, lib, multithread
token = lib.auth(tenant_id="ae5ab930-6b7b-4b0c-9a92-dcc7e8008cb9", client_id="f55ddaf6-8ebf-4474-aa00-4b06d8479c61", client_secret="9KlkgWv9WMqoDatfLINVLuJMsa057qLob+bdOj1aEv4=")

adl = core.AzureDLFileSystem(token, store_name="linxccdls02")
adl.set_expiry("Backup/{0}/{1}.csv".format(backup_folder, timestamp), "RelativeToNow", 604800000)